In [1]:
import json
import uuid
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import udahub

# Udahub Application

## Core Database

**Init DB**

In [2]:
udahub_db = "data/core/udahub.db"

In [3]:
reset_db(udahub_db)

2025-12-02 20:04:07,139 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 20:04:07,140 INFO sqlalchemy.engine.Engine COMMIT
✅ Recreated data/core/udahub.db with fresh schema


In [4]:
engine = create_engine(f"sqlite:///{udahub_db}", echo=False)
udahub.Base.metadata.create_all(bind=engine)

**Account**

In [5]:
account_id = "cultpass"
account_name = "CultPass Card"

In [6]:
with get_session(engine) as session:
    account = udahub.Account(
        account_id=account_id,
        account_name=account_name,
    )
    session.add(account)

## Integrations

**Knowledge Base**

In [7]:
cultpass_articles = []

with open('data/external/cultpass_articles.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_articles.append(json.loads(line))

In [8]:
cultpass_articles

[{'title': 'How to Reserve a Spot for an Event',
  'content': 'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping \'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
  'tags': 'reservation, events, booking, attendance'},
 {'title': "What's Included in a CultPass Subscription",
  'content': 'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee (visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass s

**Use LLM to Generate Sample Data**

In [9]:
import os
from dotenv import load_dotenv
from typing import List
from pydantic import BaseModel, Field
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_base_url = "https://openai.vocareum.com/v1"

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    base_url="https://openai.vocareum.com/v1",
    api_key=OPENAI_API_KEY,
)

Samping Paramters

In [10]:
num_articles = 15
sample_tags = [ article['tags'].split(', ') for article in cultpass_articles ]
sample_tags = set([ item for subtags in sample_tags for item in subtags ])
sample_tags_str = ", ".join(sample_tags)

In [ ]:
# TODO: Create additional 10 articles
class CultpassArticlesSchema(BaseModel):
    title: str = Field(description="Title of the article.")
    content: str = Field(description="Content of the article.")
    tags: str = Field(description="Tags of the article")

class CultpassArticlesListSchema(BaseModel):
    samples: List[CultpassArticlesSchema] = Field(description="List of CultpassArticles")

sample_articles_template = """
    You are an helpful expert creating sample of python sample article data based on given examples.  Do not provide any additional description or explanation.
    There is a mock-up project, it has the following description:

    You’ve joined a fast-growing AI startup building the next frontier in customer support automation. Your team is responsible for building UDA-Hub, 
    a Universal Decision Agent designed to plug into existing customer support systems (Zendesk, Intercom, Freshdesk, internal CRMs) and intelligently 
    resolve tickets. You need to generate sample knowledge articles in area including: {sample_tags}.  
    
    The knowledge areas given above will be used as tags for the generated articles.

    The first UDA-Hub customer Cultpass has the following sample articles to be used as knowledge to be used by the system:
    {samples}

    Provide {num_articles} more example like above 
    """

sample_articles_prompt = PromptTemplate(
    template = sample_articles_template,
    input_variables = ["samples","num_articles","sample_tags"],
    ).invoke({"samples": cultpass_articles,
              "num_articles": num_articles,
              "sample_tags": sample_tags}).to_messages()

new_cultpass_articles = llm.with_structured_output(CultpassArticlesListSchema).invoke(sample_articles_prompt).model_dump()['samples']
cultpass_articles = cultpass_articles + new_cultpass_articles

In [12]:
cultpass_articles

[{'title': 'How to Reserve a Spot for an Event',
  'content': 'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping \'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
  'tags': 'reservation, events, booking, attendance'},
 {'title': "What's Included in a CultPass Subscription",
  'content': 'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee (visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass s

In [13]:
new_sample_tags = [ article['tags'].split(', ') for article in cultpass_articles ]
new_sample_tags = set([ item for subtags in new_sample_tags for item in subtags ])
new_sample_tags_str = ", ".join(new_sample_tags)

In [14]:
new_sample_tags

{'access',
 'attendance',
 'benefits',
 'billing',
 'booking',
 'cancelation',
 'escalation',
 'events',
 'login',
 'password',
 'pause',
 'pricing',
 'reservation',
 'subscription',
 'support'}

In [15]:
sample_tags

{'access',
 'attendance',
 'benefits',
 'billing',
 'booking',
 'cancelation',
 'escalation',
 'events',
 'login',
 'password',
 'pause',
 'pricing',
 'reservation',
 'subscription'}

Extract Tags for Sample Ticket Generation

In [16]:
if len(cultpass_articles) < 14:
    raise AssertionError("You should load the articles with at least 14 records")

In [17]:
with get_session(engine) as session:
    kb = []
    for article in cultpass_articles:
        knowledge = udahub.Knowledge(
            article_id=str(uuid.uuid4()),
            account_id=account_id,
            title=article["title"],
            content=article["content"],
            tags=article["tags"]
        )
        kb.append(knowledge)
    session.add_all(kb) 
    

**Ticket**

In [18]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [19]:
ticket_info = {
    "status": "open",
    "content": "I can't log in to my Cultpass account.",
    "owner_id": cultpass_users[0]["id"],
    "owner_name": cultpass_users[0]["name"],
    "role": "user",
    "channel": "chat",
    "tags": "login, access",
}

In [20]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()

    if not user:
        user = udahub.User(
            user_id=str(uuid.uuid4()),
            account_id=account_id,
            external_user_id=ticket_info["owner_id"],
            user_name=ticket_info["owner_name"],
        )
    
    ticket = udahub.Ticket(
        ticket_id=str(uuid.uuid4()),
        account_id=account_id,
        user_id=user.user_id,
        channel=ticket_info["channel"],
    )
    metadata = udahub.TicketMetadata(
        ticket_id=ticket.ticket_id,
        status=ticket_info["status"],
        main_issue_type=None,
        tags=ticket_info["tags"],
    )

    first_message = udahub.TicketMessage(
        message_id=str(uuid.uuid4()),
        ticket_id=ticket.ticket_id,
        role=ticket_info["role"],
        content=ticket_info["content"],
    )

    session.add_all([user, ticket, metadata, first_message])


# Tests

In [21]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    print(account)

<Account(account_id='cultpass', account_name='CultPass Card')>


In [22]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    for article in account.knowledge_articles:
        print(article)

<Knowledge(article_id='0a4bb7b2-d32a-4bc2-971c-4a29f1108946', title='How to Reserve a Spot for an Event')>
<Knowledge(article_id='d3f84798-9c95-45df-a8a5-b0a2769db7f0', title='What's Included in a CultPass Subscription')>
<Knowledge(article_id='36730cf6-2956-4411-9c06-930b9cbd108e', title='How to Cancel or Pause a Subscription')>
<Knowledge(article_id='24e91473-9edc-4915-ba92-202b3de6ac37', title='How to Handle Login Issues?')>
<Knowledge(article_id='1ea690cb-e550-4779-a803-82e3cfca3605', title='How to Update Payment Information')>
<Knowledge(article_id='71f6440b-0343-4acc-881f-90545c51b601', title='How to Book a Premium Experience')>
<Knowledge(article_id='011ebfb0-114b-4953-9d22-588882047fc7', title='Understanding CultPass Pricing Tiers')>
<Knowledge(article_id='769a0cb9-fa75-4b2e-b872-a9c7022ae947', title='How to Escalate a Support Ticket')>
<Knowledge(article_id='06631426-3761-4769-95d6-a35361947281', title='How to Access Past Event History')>
<Knowledge(article_id='b3dd1fed-4e21-4

In [23]:
with get_session(engine) as session:
    users = session.query(udahub.User).all()
    for user in users:
        print(user)

<User(user_id='bf33dc7d-6fef-4cb1-b96e-168f57dc2295', user_name='Alice Kingsley', external_user_id='a4ab87')>


In [24]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    ticket:udahub.Ticket = user.tickets[0]
    for message in ticket.messages:
        print(message)

<TicketMessage(message_id='6e3ddefd-caf8-41da-b516-5ea6cd192590', role='user', content='I can't log in to my Cultpass ...')>
